In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
#authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules 

In [0]:
#https://drive.google.com/open?id=1kfy_mlM_9DqcM-nGxlSuX5_bISqhm-P7
myfile = drive.CreateFile({'id': '1kfy_mlM_9DqcM-nGxlSuX5_bISqhm-P7'})
myfile.GetContentFile('Online Retail.xlsx')

In [76]:
data = pd.read_excel("Online Retail.xlsx")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [77]:
# Exploring the columns of the data 
data.columns 

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [78]:
# Exploring the different regions of transactions 
data.Country.unique() 

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

Cleaning the Data

In [0]:
# Stripping extra spaces in the description 
data['Description'] = data['Description'].str.strip() 

In [0]:
# Dropping the rows without any invoice number 
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 

In [0]:
# Dropping all transactions which were done on credit 
data = data[~data['InvoiceNo'].str.contains('C')] 

Splitting the data according to the region of transaction

In [0]:
# Transactions done in France 
basket_France = (data[data['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [0]:
# Transactions done in the United Kingdom 
basket_UK = (data[data['Country'] =="United Kingdom"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [0]:
# Transactions done in Portugal 
basket_Por = (data[data['Country'] =="Portugal"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [0]:
# Transactions done in Sweden
basket_Sweden = (data[data['Country'] =="Sweden"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

Hot encoding the Data

In [0]:
# Defining the hot encoding function to make the data suitable  
# for the concerned libraries 
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

In [0]:
# Encoding the datasets 
basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 
  
basket_encoded = basket_UK.applymap(hot_encode) 
basket_UK = basket_encoded 
  
basket_encoded = basket_Por.applymap(hot_encode) 
basket_Por = basket_encoded 
  
basket_encoded = basket_Sweden.applymap(hot_encode) 
basket_Sweden = basket_encoded 

Buliding the models and analyzing the results

1) France:

In [0]:
# Building the model 
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True) 

In [89]:
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                                           antecedents  ... conviction
44                        (JUMBO BAG WOODLAND ANIMALS)  ...        inf
259  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...  ...        inf
270  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...  ...        inf
301  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...  ...  34.897959
302  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...  ...  34.489796

[5 rows x 9 columns]


From the above output, it can be seen that paper cups and paper and plates are bought together in France. This is because the French have a culture of having a get-together with their friends and family atleast once a week. Also, since the French government has banned the use of plastic in the country, the people have to purchase the paper -based alternatives.

---------------------------------------------------------------------

2) United Kingdom:

In [0]:
# Building the model
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True) 

In [91]:
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                                       antecedents  ... conviction
117           (BEADED CRYSTAL HEART PINK ON STICK)  ...  39.637371
2020  (JAM MAKING SET PRINTED, SUKI  SHOULDER BAG)  ...  26.096206
2295         (HERB MARKER MINT, HERB MARKER THYME)  ...  21.947227
2301   (HERB MARKER ROSEMARY, HERB MARKER PARSLEY)  ...  20.444951
2302      (HERB MARKER THYME, HERB MARKER PARSLEY)  ...  20.443842

[5 rows x 9 columns]


If the rules for British transactions are analyzed a little deeper, it is seen that the British people buy different coloured tea-plates together. A reason behind this may be because typically the British enjoy tea very much and often collect different coloured tea-plates for different ocassions.

-----------------------------------------------------------------

 3) Portugal:

In [0]:
# Building the model
frq_items = apriori(basket_Por, min_support = 0.05, use_colnames = True) 

In [93]:
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                             antecedents  ... conviction
1170    (SET 12 COLOUR PENCILS SPACEBOY)  ...        inf
1171  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf
1172  (SET OF 4 KNICK KNACK TINS LONDON)  ...        inf
1173  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf
1174  (SET 12 COLOUR PENCILS DOLLY GIRL)  ...        inf

[5 rows x 9 columns]


On analyzing the association rules for Portuguese transactions, it is observed that Tiffin sets (Knick Knack Tins) and colour pencils. These two products typically belong to a primary school going kid. These two products are required by children in school to carry their lunch and for creative work respectively and hence are logically make sense to be paired together.

---------------------------------------------------------------------

4) Sweden:

In [0]:
# Building the model
frq_items = apriori(basket_Sweden, min_support = 0.05, use_colnames = True) 

In [95]:
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

                           antecedents  ... conviction
0        (12 PENCILS SMALL TUBE SKULL)  ...        inf
1        (PACK OF 72 SKULL CAKE CASES)  ...        inf
4              (36 DOILIES DOLLY GIRL)  ...        inf
5       (ASSORTED BOTTLE TOP  MAGNETS)  ...        inf
180  (CHILDRENS CUTLERY CIRCUS PARADE)  ...        inf

[5 rows x 9 columns]


On analyzing the above rules for Sweden, it is found that boys’ and girls’ cutlery are paired together. This makes practical sense because when a parent goes shopping for cutlery for his/her children, he/she would want the product to be a little customized according to the kid’s wishes.